## 1. ConversationBufferMemory

**이전 대화 내용**을 기억하고 참조할 수 있도록 돕습니다.
 즉, 사용자와 모델 간의 대화 내용을 그대로 저장하고 필요할 때 불러와서 모델이 문맥을 이해하고 일관성 있는 답변을 생성하는 데 사용됩니다.

 ※ 모든 이전 대화 내용을 처음부터 끝까지 저장

In [ ]:
from langchain.memory import ConversationBufferMemory

`save_context(inputs, outputs)` 메서드를 사용하여 대화 기록을 저장할 수 있습니다.

- 이 메서드는 inputs와 outputs 두 개의 인자를 받습니다.
- inputs는 사용자의 입력을, outputs는 AI의 출력을 저장합니다.
- 이 메서드를 사용하면 대화 기록이 **history 키**에 저장됩니다.
- 이후 `load_memory_variables` 메서드를 사용하여 저장된 대화 기록을 확인할 수 있습니다.

In [ ]:
memory = ConversationBufferMemory()
memory.save_context(
    inputs={
        "human": "안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?"
    },
    outputs={
        "ai": "안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?"
    },
)

In [ ]:
# 'history' 키에 저장된 대화 기록을 확인합니다.
memory.load_memory_variables({})

In [ ]:
# history에 저장된 대화 기록을 확인합니다.
print(memory.load_memory_variables({})["history"])

**return_messages=True** 로 설정하면 HumanMessage 와 AIMessage 객체를 반환합니다.

In [ ]:
memory = ConversationBufferMemory(return_messages=True)

memory.save_context(
    inputs={
        "human": "안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?"
    },
    outputs={
        "ai": "안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?"
    },
)

memory.save_context(
    inputs={"human": "네, 신분증을 준비했습니다. 이제 무엇을 해야 하나요?"},
    outputs={
        "ai": "감사합니다. 신분증 앞뒤를 명확하게 촬영하여 업로드해 주세요. 이후 본인 인증 절차를 진행하겠습니다."
    },
)

memory.load_memory_variables({})["history"]

### Chain 에 적용

In [ ]:
!pip install langchain-teddynote

In [ ]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain

# LLM 모델을 생성합니다.
llm = ChatOpenAI(temperature=0)

# ConversationChain을 생성!!
conversation = ConversationChain(
    # ConversationBufferMemory를 사용합니다.
    llm=llm,
    memory=ConversationBufferMemory(),
)

In [ ]:
# 대화를 시작합니다.
response = conversation.predict(
    input="안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?"
)
print(response)


이전의 대화 기록을 기억하고 있는지 확인합니다.

In [ ]:
# 이전 대화내용을 불렛포인트로 정리해 달라는 요청을 보냅니다.
response = conversation.predict(
    input="이전 답변을 불렛포인트 형식으로 정리하여 알려주세요."
)
print(response)

## 02. 대화 버퍼 윈도우 메모리(ConversationBufferWindowMemory)

Q. ConversationBuffer'Window'Memory는 ConversationBufferMemory랑 뭐가 달라? window만 추가됐는데

A. 가장 큰 차이점은 최근 N개의 대화 턴(turn)만 기억한다는 점입니다. 여기서 'window'는 바로 이 '최근 N개'를 의미합니다.

Q. 제미나이나 chatGPT는 ConversationBufferWindowMemory 방식을 사용해?

A.대규모 언어 모델(LLM)들은 개념적으로 ConversationBufferWindowMemory와 유사한 '컨텍스트 윈도우(Context Window)' 또는 **'토큰 한도(Token Limit)'**라는 것을 가지고 있습니다.

제미나이, ChatGPT 같은 모델들은 한 번에 처리할 수 있는 입력(프롬프트)의 길이에 제한이 있습니다.  이 제한을 'Context Window'라고 부름.

따라서, 제미나이나 ChatGPT는 ConversationBufferWindowMemory라는 특정 구현체를 사용하는 것이 아니라, 그와 유사한 '최근 대화만 참조하는' 방식의 컨텍스트 관리 원리를 따르며, ConversationBufferWindowMemory는 이를 효과적으로 구현하기 위한 프레임워크 수준의 솔루션이라고 이해하시면 됩니다.

대화기록을 확인해 보면 최근 2개 의 메시지만 반환하는 것을 확인할 수 있습니다.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2, return_messages=True)

memory.save_context(
    inputs={
        "human": "안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?"
    },
    outputs={
        "ai": "안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?"
    },
)
memory.save_context(
    inputs={"human": "네, 신분증을 준비했습니다. 이제 무엇을 해야 하나요?"},
    outputs={
        "ai": "감사합니다. 신분증 앞뒤를 명확하게 촬영하여 업로드해 주세요. 이후 본인 인증 절차를 진행하겠습니다."
    },
)
memory.save_context(
    inputs={"human": "사진을 업로드했습니다. 본인 인증은 어떻게 진행되나요?"},
    outputs={
        "ai": "업로드해 주신 사진을 확인했습니다. 이제 휴대폰을 통한 본인 인증을 진행해 주세요. 문자로 발송된 인증번호를 입력해 주시면 됩니다."
    },
)

# 대화 기록을 확인합니다.
memory.load_memory_variables({})["history"]


## 03. 대화 토큰 버퍼 메모리(ConversationTokenBufferMemory)

대화의 개수가 아닌 **토큰 길이** 를 사용하여 대화내용을 플러시(flush, 오래된 내용 버리기)할 시기를 결정합니다.

max_token_limit: 대화 내용을 저장할 최대 토큰의 길이

In [ ]:
from langchain.memory import ConversationTokenBufferMemory
from langchain_openai import ChatOpenAI

# LLM 모델 생성
llm = ChatOpenAI()

# 메모리 설정
memory = ConversationTokenBufferMemory(
    llm=llm, max_token_limit=150, return_messages=True
    # 최대 토큰 길이를 150개로 제한
)

In [ ]:
memory.save_context(
    inputs={
        "human": "안녕하세요, 저는 최근에 여러분 회사의 공작 기계를 구매했습니다. 설치 방법을 알려주실 수 있나요?"
    },
    outputs={
        "ai": "안녕하세요! 구매해 주셔서 감사합니다. 해당 기계 모델 번호를 알려주시겠어요?"
    },
)
memory.save_context(
    inputs={"human": "네, 모델 번호는 XG-200입니다."},
    outputs={
        "ai": "감사합니다. XG-200 모델의 설치 안내를 도와드리겠습니다. 먼저, 설치할 장소의 전원 공급 상태를 확인해주세요. 기계는 220V 전원이 필요합니다."
    },
)
memory.save_context(
    inputs={"human": "전원은 확인했습니다. 다음 단계는 무엇인가요?"},
    outputs={
        "ai": "좋습니다. 다음으로, 기계를 평평하고 안정된 바닥에 배치해 주세요. 이후, 제공된 사용자 매뉴얼에 따라 케이블 연결을 진행해 주시기 바랍니다."
    },
)

# 대화내용을 확인합니다.
memory.load_memory_variables({})["history"]

## 04. 대화 엔티티 메모리(ConversationEntityMemory) 💛

엔티티 메모리는 엔티티에 대한 정보를 추출하고(LLM 사용) 시간이 지남에 따라 해당 엔티티에 대한 지식을 축적합니다(역시 LLM 사용).

 **'엔티티(Entity)'**는 대화에서 등장하는 특정 '개념' 또는 **'주체'**를 의미합니다. ->'사람 이름', '회사 이름', '제품명', '장소' 등

[사례]
1. 애플 Siri, 구글 어시스턴트, 아마존 Alexa (음성 비서):

  - 엔티티: 사용자 자신, 가족 구성원, 즐겨 듣는 음악 장르, 선호하는 뉴스 채널, 집 주소, 직장 주소 등입니다.
  - 활용: 예를 들어, 사용자가 "엄마에게 전화 걸어줘"라고 했을 때, 음성 비서는 '엄마'가 누구인지, 연락처가 무엇인지 엔티티 메모리에서 찾아 전화를 겁니다.
  
2. 넷플릭스, 유튜브 (콘텐츠 추천 시스템):
- 엔티티: 사용자, 시청한 콘텐츠, 선호하는 장르, 배우, 감독 등입니다.
- 활용: 비록 직접적인 대화는 아니지만, 사용자가 어떤 콘텐츠를 시청하고 '좋아요'를 누르거나 검색했는지 등의 행동 데이터를 기반으로 '사용자'라는 엔티티의 취향을 학습합니다.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory

from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
# -> Entity 메모리를 효과적으로 사용하기 위하여, 제공되는 프롬프트를 사용합니다.

In [ ]:
# Entity Memory를 사용하는 프롬프트 내용을 출력합니다.
print(ENTITY_MEMORY_CONVERSATION_TEMPLATE.template)

Context: {entities} 변수를 통해 사용자가 제공한 개인화된 정보나 특정 데이터를 활용할 수 있도록 공간을 마련합니다.

Current conversation: {history} 변수를 통해 이전 대화 내용을 기억하고 문맥을 이해하도록 합니다.

Last line: {input} 변수를 통해 사용자의 현재 질문을 받아들이도록 설정합니다.

In [ ]:
# LLM 을 생성합니다.
llm = ChatOpenAI(temperature=0)

# ConversationChain 을 생성합니다.
conversation = ConversationChain(
    llm=llm,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE, #위 템플릿 사용
    memory=ConversationEntityMemory(llm=llm),
)

In [ ]:
conversation.predict(
    input="테디와 셜리는 한 회사에서 일하는 동료입니다."
    "테디는 개발자이고 셜리는 디자이너입니다. "
    "그들은 최근 회사에서 일하는 것을 그만두고 자신들의 회사를 차릴 계획을 세우고 있습니다."
)


Entity는 memory.entity_store.store 에서 확인할 수 있습니다.

In [ ]:
# entity memory 를 출력합니다.
conversation.memory.entity_store.store

> **Q. 어떤걸 entity로 정할지는 어떻게 결정되는 거지?**



## 05. 대화 지식그래프 메모리(ConversationKGMemory)

KG (Knowledge Graph):
개체(entity)와 이들 간의 관계를 그래프 형태로 표현하여 지식을 구조화하는 방식

- EX) '사람-직업', '회사-제품', '도시-국가'와 같은 관계를 노드(개체)와 엣지(관계)로 연결하여 표현

In [ ]:
pip install -U langchain-community

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationKGMemory

In [ ]:
llm = ChatOpenAI(temperature=0)

memory = ConversationKGMemory(llm=llm, return_messages=True)
memory.save_context(
    {"input": "이쪽은 Pangyo 에 거주중인 김셜리씨 입니다."},
    {"output": "김셜리씨는 누구시죠?"},
)
memory.save_context(
    {"input": "김셜리씨는 우리 회사의 신입 디자이너입니다."},
    {"output": "만나서 반갑습니다."},
)


### Chain 에 메모리 활용하기

In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain

llm = ChatOpenAI(temperature=0)

template = """The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.
The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template)

conversation_with_kg = ConversationChain(
    llm=llm, prompt=prompt, memory=ConversationKGMemory(llm=llm)
)


In [ ]:
conversation_with_kg.predict(
    input="My name is Teddy. Shirley is a coworker of mine, and she's a new designer at our company."
)

In [ ]:
# Shirley 에 대한 질문
conversation_with_kg.memory.load_memory_variables({"input": "who is Shirley?"})

{'history': 'On Shirley: Shirley is a coworker. Shirley is a new designer. Shirley is at company.'}

## 06. 대화 요약 메모리(ConversationSummaryMemory)

시간 경과에 따른 **대화의 요약**을 생성

이 메모리는 과거 메시지 기록을 프롬프트에 그대로 보관하면 토큰을 너무 많이 차지할 수 있는 긴 대화에 가장 유용합니다.

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain_openai import ChatOpenAI

memory = ConversationSummaryMemory(
    llm=ChatOpenAI(temperature=0), return_messages=True)


In [ ]:
memory.save_context(
    inputs={"human": "유럽 여행 패키지의 가격은 얼마인가요?"},
    outputs={
        "ai": "유럽 14박 15일 패키지의 기본 가격은 3,500유로입니다. 이 가격에는 항공료, 호텔 숙박비, 지정된 관광지 입장료가 포함되어 있습니다. 추가 비용은 선택하신 옵션 투어나 개인 경비에 따라 달라집니다."
    },
)
memory.save_context(
    inputs={"human": "여행 중에 방문할 주요 관광지는 어디인가요?"},
    outputs={
        "ai": "이 여행에서는 파리의 에펠탑, 로마의 콜로세움, 베를린의 브란덴부르크 문, 취리히의 라이네폴 등 유럽의 유명한 관광지들을 방문합니다. 각 도시의 대표적인 명소들을 포괄적으로 경험하실 수 있습니다."
    },
)
memory.save_context(
    inputs={"human": "여행자 보험은 포함되어 있나요?"},
    outputs={
        "ai": "네, 모든 여행자에게 기본 여행자 보험을 제공합니다. 이 보험은 의료비 지원, 긴급 상황 발생 시 지원 등을 포함합니다. 추가적인 보험 보장을 원하시면 상향 조정이 가능합니다."
    },
)
memory.save_context(
    inputs={
        "human": "항공편 좌석을 비즈니스 클래스로 업그레이드할 수 있나요? 비용은 어떻게 되나요?"
    },
    outputs={
        "ai": "항공편 좌석을 비즈니스 클래스로 업그레이드하는 것이 가능합니다. 업그레이드 비용은 왕복 기준으로 약 1,200유로 추가됩니다. 비즈니스 클래스에서는 더 넓은 좌석, 우수한 기내식, 그리고 추가 수하물 허용량 등의 혜택을 제공합니다."
    },
)


저장된 메모리의 history에서 이전의 모든 대화를 압축적으로 요약한 내용을 확인할 수 있습니다.

In [ ]:
# 저장된 메모리 확인
print(memory.load_memory_variables({})["history"])

## ConversationSummaryBufferMemory

buffer + ConversationSummaryMemory

최근 대화내용의 버퍼를 메모리에 유지하되, 이전 대화내용을 완전히 플러시(flush)하지 않고 요약으로 컴파일하여 두 가지를 모두 사용합니다.

대화내용을 플러시할 시기를 결정하기 위해 상호작용의 개수가 아닌 토큰 길이 를 사용합니다.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory

llm = ChatOpenAI()

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=200,  # 요약의 기준이 되는 토큰 길이를 설정합니다.
    return_messages=True,
)


In [ ]:
memory.save_context(
    inputs={"human": "유럽 여행 패키지의 가격은 얼마인가요?"},
    outputs={
        "ai": "유럽 14박 15일 패키지의 기본 가격은 3,500유로입니다. 이 가격에는 항공료, 호텔 숙박비, 지정된 관광지 입장료가 포함되어 있습니다. 추가 비용은 선택하신 옵션 투어나 개인 경비에 따라 달라집니다."
    },
)


In [ ]:
# 메모리에 저장된 대화내용 확인
memory.load_memory_variables({})["history"]

아직은 대화내용을 요약하지 않습니다. 기준이 되는 200 토큰에 도달하지 않았기 때문입니다.

대화를 추가로 저장하여 200 토큰 제한을 넘기도록 해 보겠습니다.

In [ ]:
memory.save_context(
    inputs={"human": "여행 중에 방문할 주요 관광지는 어디인가요?"},
    outputs={
        "ai": "이 여행에서는 파리의 에펠탑, 로마의 콜로세움, 베를린의 브란덴부르크 문, 취리히의 라이네폴 등 유럽의 유명한 관광지들을 방문합니다. 각 도시의 대표적인 명소들을 포괄적으로 경험하실 수 있습니다."
    },
)
memory.save_context(
    inputs={"human": "여행자 보험은 포함되어 있나요?"},
    outputs={
        "ai": "네, 모든 여행자에게 기본 여행자 보험을 제공합니다. 이 보험은 의료비 지원, 긴급 상황 발생 시 지원 등을 포함합니다. 추가적인 보험 보장을 원하시면 상향 조정이 가능합니다."
    },
)

In [ ]:
# 메모리에 저장된 대화내용 확인
memory.load_memory_variables({})["history"]

-> 가장 최근 1개의 대화에 대해서는 요약이 진행되지 않지만, 이전의 대화내용은 요약본으로 저장되어 있습니다.

##  07. 벡터저장소 검색 메모리(VectorStoreRetrieverMemory)
VectorStore: 텍스트 데이터를 숫자의 배열인 '임베딩(Embedding)' 형태로 변환하여 저장하고 관리하는 데이터베이스입니다. 의미가 비슷한 텍스트는 벡터 공간에서 서로 가까운 위치!

 벡터 스토어에 메모리를 저장하고 호출될 때마다 '현재 대화 내용과 유사도 점수가 가장 높은 상위 K개'의 문서를 쿼리합니다.

** 대화내용의 순서를 명시적으로 추적하지 않는다**는 점 에서 다른 대부분의 메모리 클래스와 다릅니다.

 장점:
 - 토큰 효율성: 모든 대화를 LLM에 전달할 필요 없이 관련 있는 소수의 정보만 전달하므로, API 비용을 절감하고 응답 속도를 향상시킬 수 있습니다.
 - 확장성: 대화가 아무리 길어져도 모든 내용을 LLM의 컨텍스트 윈도우에 넣을 필요가 없으므로, 매우 장기적인 대화나 복잡한 지식 기반을 가진 챗봇에 적합합니다. 메모리 저장 공간의 제약이 거의 없고, LLM의 토큰 제한을 효과적으로 관리할 수 있습니다.

 ->매우 긴 대화 관리: 대화의 길이에 거의 제한 없이 과거 정보를 활용할 수 있습니다.

 단점:

- 구현 복잡성: Vector Store를 설정하고 관리해야 하므로 다른 메모리 타입보다 설정이 복잡할 수 있습니다.
- 정보 손실 가능성: 현재 대화와 의미적으로 관련성이 낮다고 판단된 정보는 검색되지 않아 LLM에게 전달되지 않을 수 있습니다.
- 비용: 임베딩 생성 및 Vector Store 운영에 비용이 발생할 수 있습니다.

먼저, 벡터 스토어를 초기화 합니다.

In [ ]:
!pip install faiss-cpu

In [ ]:
import faiss
from langchain_openai import OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS


# 임베딩 모델을 정의합니다.
embeddings_model = OpenAIEmbeddings()

# Vector Store 를 초기화 합니다.
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})


## 08. LCEL Chain 에 메모리 추가

LangChain Expression Language (LCEL) Chain: 파이프 연산자(|)**를 사용하여 연결 -> 앞에서 배운거

### LCEL (대화내용 기억하기): 메모리 추가
임의의 체인에 메모리를 추가하는 방법을 보여줍니다. 현재 메모리 클래스를 사용할 수 있지만 수동으로 연결해야 합니다

'현재 메모리 클래스를 사용할 수 있지만': ConversationEntityMemory 등 LangChain에서 제공하는 다양한 메모리 클래스들이 준비되어 있고, 여러분이 필요에 따라 이 클래스들을 가져와서 사용할 수 있다는 뜻입니다. 예를 들어, memory = ConversationBufferMemory() 처럼요.

'수동으로 연결해야 합니다': 이렇게 만들어진 memory 객체를 LangChain의 Chain에 코드 상으로 명시적으로 연결(pass)해 주어야 한다는 의미입니다. 개발자가 직접 ConversationChain(llm=llm, memory=memory) 와 같이 memory 파라미터를 통해 연결해 주어야 합니다.

In [ ]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI


# ChatOpenAI 모델을 초기화합니다.
model = ChatOpenAI()

# 대화형 프롬프트를 생성합니다. 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)


대화내용을 저장할 메모리인 ConversationBufferMemory 생성하고 return_messages 매개변수를 True로 설정하여, 생성된 인스턴스가 메시지를 반환하도록 합니다.

memory_key 설정: 추후 Chain 의 prompt 안에 대입될 key 입니다. 변경하여 사용할 수 있습니다.

In [ ]:
# 대화 버퍼 메모리를 생성하고, 메시지 반환 기능을 활성화합니다.
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")


In [ ]:
memory.load_memory_variables({})  # 메모리 변수를 빈 딕셔너리로 초기화합니다.
# 아직 저장하지 않았으므로, 대화기록은 비어 있습니다.

RunnablePassthrough.assign을 사용하여 chat_history 변수에 memory.load_memory_variables 함수의 결과를 할당하고, 이 결과에서 chat_history 키에 해당하는 값을 추출합니다.

In [ ]:
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables)
    | itemgetter("chat_history")  # memory_key 와 동일하게 입력합니다.
)
runnable.invoke({"input": "hi"})

Q. {'input': 'hi', 'chat_history': {'chat_history': []}} -> 왜 다르게 나왔지?

In [ ]:
runnable.invoke({"input": "hi"})

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)


runnable 에 첫 번째 대화를 시작합니다.

- input: 사용자 입력 대화가 전달됩니다.
- chat_history: 대화 기록이 전달됩니다.

In [ ]:
runnable.invoke({"input": "hi!"})

In [ ]:
chain = runnable | prompt | model
# chain 객체의 invoke 메서드를 사용하여 입력에 대한 응답을 생성합니다.
response = chain.invoke({"input": "만나서 반갑습니다. 제 이름은 임가영입니다."})
print(response.content)  # 생성된 응답을 출력합니다.

In [ ]:
memory.load_memory_variables({})

memory.save_context 함수는 입력 데이터(inputs)와 응답 내용(response.content)을 메모리에 저장하는 역할을 합니다. 이는 AI 모델의 학습 과정에서 현재 상태를 기록하거나, 사용자의 요청과 시스템의 응답을 추적하는 데 사용될 수 있습니다.

In [ ]:
# 입력된 데이터와 응답 내용을 메모리에 저장합니다.
memory.save_context(
    {"human": "만나서 반갑습니다. 제 이름은 임가영입니다."}, {"ai": response.content}
)

# 저장된 대화기록을 출력합니다.
memory.load_memory_variables({})


In [ ]:
# 이름을 기억하고 있는지 추가 질의합니다.
response = chain.invoke({"input": "제 이름이 무엇이었는지 기억하세요?"})
# 답변을 출력합니다.
print(response.content)


## 09. SQLite에 대화내용 저장

SQLite 이외의 데이터베이스와 함께 사용하려면 해당 데이터베이스 드라이버를 설치해야 합니다.

사용방법

storage를 사용하려면 다음 2가지만 제공하면 됩니다:

1. session_id - 사용자 이름, 이메일, 채팅 ID 등과 같은 세션의 고유 식별자입니다.

2. connection - 데이터베이스 연결을 지정하는 문자열입니다. 이 문자열은 SQLAlchemy의 create_engine 함수에 전달됩니다.

In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# SQLChatMessageHistory 객체를 생성하고 세션 ID와 데이터베이스 연결 파일을 설정
chat_message_history = SQLChatMessageHistory(
    session_id="sql_history", connection="sqlite:///sqlite.db"
)


## 10. RunnableWithMessageHistory에 ChatMessageHistory추가

RunnableWithMessageHistory는 **LCEL(LangChain Expression Language) 체인에 대화 기록(Chat History) 기능을 쉽게 추가할 수 있도록 해주는 유틸리티 래퍼(wrapper)**입니다. 즉, LCEL을 사용하여 구성된 Chain에 ChatMessageHistory 객체를 연동하여, 대화의 상태(기억)를 관리하는 기능을 제공합니다.

### 이전 대화를 기억하는 Chain 생성방법

1. 이전 대화내용을 기억하는 multi-turn Chain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


# 프롬프트 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 Question-Answering 챗봇입니다. 주어진 질문에 대한 답변을 제공해주세요.",
        ),
        # 대화기록용 key 인 chat_history 는 가급적 변경 없이 사용하세요!
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "#Question:\n{question}"),  # 사용자 입력을 변수로 사용
    ]
)

# llm 생성
llm = ChatOpenAI(model_name="gpt-4o")

# 일반 Chain 생성
chain = prompt | llm | StrOutputParser()
